In [36]:
%matplotlib inline

import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.cross_validation import StratifiedKFold
from sklearn.naive_bayes import *
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import recall_score

pd.set_option('display.max_columns', None)

In [37]:
beauty = pd.read_csv("../data/beauty5k.csv")
beauty = beauty.dropna(subset=["reviewText", "overall"])

# encode sentiment to "pos" and "neg"
# remove punctuation and tokenize reviewText

def remove_punc(s):
    return s.translate(None, string.punctuation)

def low(s):
    return s.lower()

def lem(tokens):
    lemmatizer = nltk.WordNetLemmatizer()
    lem_tokens = []
    for token in tokens:
        lem_tokens.append(lemmatizer.lemmatize(token))
    return lem_tokens

beauty["sentiment"] = beauty["overall"].apply(lambda x : 'pos' if x > 3 else 'neg')
beauty["token"] = beauty["reviewText"].apply(low).apply(remove_punc).apply(word_tokenize)
beauty["token"] = beauty["token"].apply(lem)

# POS tag
beauty["tagged_token"] = beauty["token"].apply(nltk.pos_tag)

# remove stop words
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

def token_remove_stop(token_list):
    return [token for token in token_list if token not in stop]

def tagged_token_remove_stop(list_tuple):
    return [(token, tag) for (token, tag) in list_tuple if token not in stop]

beauty["token_no_stop"] = beauty["token"].apply(token_remove_stop)
beauty["tagged_token"] = beauty["tagged_token"].apply(tagged_token_remove_stop)

print beauty.head(2)

   Unnamed: 0      reviewerID        asin reviewerName helpful  \
0       27677  A1ZNEYW8GJIF1P  B00006IV2F     Pat Sims  [0, 0]   
1       27716  A37V39UMYM2JIJ  B00006IV2F   rescueAdog  [1, 1]   

                                          reviewText  overall    summary  \
0  Works well, no burning  fingers or neck like a...      5.0  Value buy   
1  Finally found a curling iron that does all tha...      5.0    Perfect   

   unixReviewTime   reviewTime sentiment  \
0    1.382832e+09  10 27, 2013       pos   
1    1.329264e+09  02 15, 2012       pos   

                                               token  \
0  [work, well, no, burning, finger, or, neck, li...   
1  [finally, found, a, curling, iron, that, doe, ...   

                                        tagged_token  \
0  [(work, NN), (well, RB), (burning, NN), (finge...   
1  [(finally, RB), (found, VBD), (curling, NN), (...   

                                       token_no_stop  
0  [work, well, burning, finger, neck, like, c

In [38]:
review_full = []
for r in beauty["token_no_stop"]:
    review_full.append(' '.join(r))
print review_full[1]

finally found curling iron doe promise great price adjustable heat good burning hair finger hair ha never looked better somehow also curl hair look thicker brush help hold fine hair without slipping like others


In [39]:
sent_full = beauty["sentiment"]

In [40]:
Data_train, Data_test, Train_labels, Test_labels = train_test_split(review_full, sent_full, test_size=0.2, random_state=111)

In [41]:
len(Data_train)

3999

### convert to tfidf matrix

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer()

tr_features_all = tfv.fit_transform(Data_train)
te_features_all = tfv.transform(Data_test)
print tr_features_all.shape

(3999, 10723)


### Compare with and without PCA using Naive Bayes. It turns out using PCA doesn't improve the accuracy, but will definitely save time. I will use PCA in the following analysis. 

In [44]:
# without PCA
model = BernoulliNB(binarize=0.5)
model.fit(tr_features_all, Train_labels)
acc_nopca = metrics.accuracy_score(model.predict(te_features_all),Test_labels)
print acc_nopca

0.602


In [45]:
# with PCA
svd = TruncatedSVD(n_components=200)
tr_features_truncated = svd.fit_transform(tr_features_all)
te_features_truncated = svd.transform(te_features_all)
print tr_features_truncated.shape

model.fit(tr_features_truncated, Train_labels)
acc_pca = metrics.accuracy_score(model.predict(te_features_truncated),Test_labels)
print acc_pca

(3999, 200)
0.601


### Instead of Bernoulli distribution, tf-idf fits in Multinomial distribution better. PCAed features contain negative values, which is not allowed in the method. So here I use the full feature matrix.

In [47]:
model = MultinomialNB()
model.fit(tr_features_all, Train_labels)
print metrics.accuracy_score(model.predict(te_features_all),Test_labels)

0.679


### Another way to reduce dimension is to use top frequent words instead of the whole vocabulary.

In [48]:
t = []
for line in Data_train:
    l = nltk.word_tokenize(line)
    for w in l:
        t.append(w)
word_features = nltk.FreqDist(t)
print len(word_features)

10746


In [61]:
topwords = [fpair[0] for fpair in list(word_features.most_common(len(word_features)/8))]
tfv.fit(topwords)
tr_features = tfv.transform(Data_train)
te_features = tfv.transform(Data_test)

# model = BernoulliNB(binarize=0.5)
model = MultinomialNB()
model.fit(tr_features, Train_labels)
print  metrics.accuracy_score(model.predict(te_features),Test_labels)

0.743


### Try different algorithms

In [66]:
models = {'MultinomialNB':MultinomialNB(),
          'Logistic' : linear_model.LogisticRegression(C=10),
          'Decision Tree' : DecisionTreeClassifier(random_state=20160121, criterion='entropy'),
          'Perceptron': linear_model.Perceptron(n_iter=1000)}
# trained_models = {}
for name, model in models.items():
    model.fit(tr_features, Train_labels)
#     trained_models[name] = model
    print name
    print metrics.accuracy_score(model.predict(te_features),Test_labels)

MultinomialNB
0.743
Perceptron
0.737
Logistic
0.782
Decision Tree
0.71


### Use higher order grams(up to trigram).

In [69]:
# from nltk.util import everygrams

# tri = []
# for line in Data_train:
#     l = nltk.word_tokenize(line)
#     for gram in list(everygrams(l, min_len=1, max_len=3)):
#         tri.append(gram)
# word_features_tri = nltk.FreqDist(tri)
# print len(word_features_tri)

211251


In [77]:
# topwords_tri = [fpair[0] for fpair in list(word_features_tri.most_common(len(word_features_tri)/8))]

tfv_tri = TfidfVectorizer(ngram_range=(1,3))
tr_features_tri = tfv_tri.fit_transform(Data_train)
te_features_tri = tfv_tri.transform(Data_test)

from sklearn.linear_model import SGDClassifier
models = {'MultinomialNB':MultinomialNB(),
          'Logistic' : linear_model.LogisticRegression(C=10),
          'Decision Tree' : DecisionTreeClassifier(random_state=20160121, criterion='entropy'),
          'Perceptron': linear_model.Perceptron(n_iter=1000),
          'SVM': SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=111)}
# trained_models = {}
for name, model in models.items():
    model.fit(tr_features_tri, Train_labels)
#     trained_models[name] = model
    print name
    print metrics.accuracy_score(model.predict(te_features_tri),Test_labels)

MultinomialNB
0.661
SVM
0.685
Perceptron
0.798
Logistic
0.816
Decision Tree
0.698


### Using higher order grams downgrades the performance of Naive Bayes and Decision Tree algorithms, however, it improves perceptron and logistic by a lot. The best algorithm so far is Logistic regression, giving 0.816 accuracy. It's worth noting that perceptron gives a similar accuracy. It makes me excited about the accuracy given by a much larger neural network.

### Bayes Optimization on logistic regression and SVM

In [98]:
from bayes_opt import BayesianOptimization

def log_acc(c):
    logistic = linear_model.LogisticRegression(C=c)
    logistic.fit(tr_features_tri, Train_labels)
    return metrics.accuracy_score(logistic.predict(te_features_tri),Test_labels)

logBO = BayesianOptimization(log_acc, {'c': (1, 10000)})
logBO.maximize(init_points = 10, n_iter = 300)

Initialization
-----------------------------------------
 Step |   Time |      Value |         c | 
    1 | 00m00s |    0.81900 | 1748.3434 | 
    2 | 00m00s |    0.81800 |   25.0308 | 
    3 | 00m00s |    0.81800 | 4501.0019 | 
    4 | 00m00s |    0.81800 | 8488.5667 | 
    5 | 00m00s |    0.81800 | 4049.7154 | 
    6 | 00m00s |    0.81700 | 1127.8379 | 
    7 | 00m00s |    0.81700 | 2925.9269 | 
    8 | 00m00s |    0.81800 | 8349.8761 | 
    9 | 00m00s |    0.81900 | 2104.5697 | 
   10 | 00m00s |    0.81700 | 9942.1966 | 
Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |         c | 
   11 | 00m02s |    0.81800 | 6380.0910 | 
   12 | 00m01s |    0.81800 | 7201.9655 | 
   13 | 00m01s |    0.81800 | 5439.9441 | 
   14 | 00m01s |    0.81800 | 9228.9646 | 
   15 | 00m00s |    0.81900 | 1989.9062 | 
   16 | 00m01s |    0.81900 | 1771.6175 | 
   17 | 00m01s |    0.81900 | 1802.7610 | 
   18 | 00m00s |    0.81900 | 1829.5455 | 
   19 | 00m01s |   

/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00255829]), 'nit': 5, 'funcalls': 72}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -2.58444624e-05]), 'nit': 4, 'funcalls': 52}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -1.93646915e-05]), 'nit': 5, 'funcalls': 57}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: 

   22 | 00m01s |    0.81900 | 2487.9012 | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.00584248]), 'nit': 5, 'funcalls': 72}
  " state: %s" % convergence_dict)


   23 | 00m01s |    0.81700 | 2680.9757 | 
   24 | 00m01s |    0.81800 | 7861.7068 | 


...

  308 | 00m22s |    0.81900 | 2064.8995 | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00305393]), 'nit': 7, 'funcalls': 68}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.20773132]), 'nit': 6, 'funcalls': 78}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.06389156]), 'nit': 8, 'funcalls': 94}
  " state: %s" % convergence_dict)


  309 | 00m18s |    0.81700 | 2579.7971 | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.01082918]), 'nit': 6, 'funcalls': 66}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00208452]), 'nit': 6, 'funcalls': 59}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.30875635]), 'nit': 8, 'funcalls': 78}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfg

  310 | 00m13s |    0.81900 | 2354.1464 | 


In [102]:
print logBO.res['max']

{'max_val': 0.82199999999999995, 'max_params': {'c': 72.176906970118779}}


In [107]:
def svm_acc(alpha_exp):
    alpha = np.power(10,alpha_exp)
    svm = SGDClassifier(loss='hinge', penalty='l2', alpha=alpha, n_iter=5, random_state=111)
    svm.fit(tr_features_tri, Train_labels)
    return metrics.accuracy_score(svm.predict(te_features_tri),Test_labels)

svmBO = BayesianOptimization(svm_acc, {'alpha_exp': (-4, 0)})
svmBO.maximize(init_points = 10, n_iter = 300)

print svmBO.res['max']

Initialization
-------------------------------------------
 Step |   Time |      Value |   alpha_exp | 
    1 | 00m00s |    0.81500 |     -3.7344 | 
    2 | 00m00s |    0.60100 |     -0.9987 | 
    3 | 00m00s |    0.60100 |     -0.8758 | 
    4 | 00m00s |    0.60100 |     -0.2451 | 
    5 | 00m00s |    0.60100 |     -1.0086 | 
    6 | 00m00s |    0.60100 |     -0.4837 | 
    7 | 00m00s |    0.62000 |     -2.8611 | 
    8 | 00m00s |    0.60100 |     -0.9077 | 
    9 | 00m00s |    0.60100 |     -0.5537 | 
   10 | 00m00s |    0.63100 |     -2.9023 | 
Bayesian Optimization
-------------------------------------------
 Step |   Time |      Value |   alpha_exp | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -6.72390888e-05]), 'nit': 5, 'funcalls': 57}
  " state: %s" % convergence_dict)


   11 | 00m04s |    0.81800 |     -4.0000 | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([  1.29742707e-05]), 'nit': 5, 'funcalls': 52}
  " state: %s" % convergence_dict)
/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -4.53999626e-05]), 'nit': 5, 'funcalls': 57}
  " state: %s" % convergence_dict)


   12 | 00m01s |    0.81700 |     -3.8746 | 


...

  306 | 00m12s |    0.81700 |     -3.7378 | 
  307 | 00m07s |    0.81300 |     -3.5394 | 
  308 | 00m05s |    0.60100 |     -1.2257 | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00952265]), 'nit': 5, 'funcalls': 59}
  " state: %s" % convergence_dict)


  309 | 00m06s |    0.81700 |     -3.7700 | 


/home/wuyi/.local/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.0001248]), 'nit': 5, 'funcalls': 56}
  " state: %s" % convergence_dict)


  310 | 00m16s |    0.81600 |     -3.5669 | 
{'max_val': 0.82599999999999996, 'max_params': {'alpha_exp': -3.9782093658463875}}
